In [ ]:
import psycopg2 as ps
import pandas as pd

In [ ]:
df = pd.read_csv('youtube_extract_1.csv', index_col=0) 
df.head()

In [ ]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [ ]:
host_name = 'version2.cbseqyqyanpf.us-east-2.rds.amazonaws.com'
dbname = 'v2'
port = '5432'
username = 'postgres'
password = 'Moemoney1!'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

In [ ]:
# updating the database

# checks to see if vid exists
# if does exist update row. if not, takes vid id and data then stores in temp df which will then allow me to write a new function to insert new vid ID's
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    dislike_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
            )""")

    curr.execute(create_table_command)

def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, dislike_count,comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ("""UPDATE videos
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                dislike_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_id, video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

# checks to see if vid exists
# if does exist update row. if not, takes vid id and data then stores in temp df which will then allow me to write a new function to insert new vid ID's
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count',
                                   'like_count', 'dislike_count', 'comment_count'])
    for i, row in df.iterrows():
        video_id = row.get('video_id')
        if video_id is not None and check_if_video_exists(curr, video_id):
            # Using .get() method to safely access dictionary values; it returns None if the key doesn't exist
            update_row(curr,
                       video_id,
                       row.get('video_title'),
                       row.get('view_count'),
                       row.get('like_count'),
                       row.get('dislike_count'),
                       row.get('comment_count'))
        else:
            # The video doesn't exist so we will add it to a temp df and append it using append_from_df_to_db.
            # Ensure only rows with a video_id are appended to avoid issues with missing primary key.
            if video_id is not None:
                tmp_df = tmp_df.append(row, ignore_index=True)

    return tmp_df


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")

    curr.execute(query, (video_id,))
    return curr.fetchone() is not None



def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")

    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count']
                          , row['like_count'], row['dislike_count'], row['comment_count'])

In [ ]:
# cursor method allows sql commands 
curr = conn.cursor()

In [ ]:
new_vid_df = update_db(curr,df)
conn.commit()

In [ ]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [ ]:
curr.execute("SELECT * FROM VIDEOS")
print(curr.fetchall())

In [ ]:
print(df.head())

In [ ]:
print(df.columns)

In [ ]:
'video_id' in df.columns